### Load tensorflow

In [0]:
import tensorflow as tf
# tf.reset_default_graph()
tf.random.set_seed(42)

### Collect Data
<font size="2">Download data from Project Gutenberg site -> http://www.gutenberg.org/files/1342/1342-0.txt </font>

In [0]:
!wget http://www.gutenberg.org/files/1342/1342-0.txt --quiet

In [0]:
book_text = open('1342-0.txt', encoding='utf8').read()
print('Length of the book: ' , len(book_text))

In [0]:
book_text[1000:1100]

### Tokenize the data

In [0]:
#Tokenize at character level
t = tf.keras.preprocessing.text.Tokenizer(char_level=True, lower=False)

#Fit tokenizer on the book
t.fit_on_texts(book_text)

#Vocablury size
vocab_size = len(t.word_index)
print('Number of unique characters: ', vocab_size)

In [0]:
#Convert characters in the book to Numbers
book_num = t.texts_to_sequences(book_text)

In [0]:
#Build a dictionary which can convert numbers into chars
int_to_char = dict((i,c) for c, i in t.word_index.items()) 

### Batch Generator

In [0]:
import numpy as np

In [0]:
record_num = 0 #starting batch number
sequence_length = 100 #Length of input sequence

def batch_generator(batch_size=32):
    
    #Will update batch_num
    global record_num
    
    while True:
        
        #Empty list for input and output data
        input_data = []
        output_data = []

        for i in range(batch_size):

            #input sequence
            input_seq = book_num[(record_num * sequence_length) : (record_num * sequence_length) + sequence_length]
            #Output sequence
            output_seq = book_num[(record_num * sequence_length) + sequence_length]

            input_data.append(input_seq)
            output_data.append(output_seq)
            
            record_num = record_num + 1
            
            if((record_num*sequence_length + sequence_length) > len(book_num)):
                record_num = 0
                

        #Input data one hot encoding
        input_data = tf.keras.utils.to_categorical(input_data,num_classes=vocab_size+1)

        #Output data one hot encoding
        output_data = tf.keras.utils.to_categorical(output_data,num_classes=vocab_size+1)
 
        print("len(input_data) : ",len(input_data))
        print("sequence_length: ",sequence_length)
        print("vocab_size: ",vocab_size)
        print("input_data.shape: ",input_data.shape)
        #Reshape input data into 3 dimensional numpy array
        #batch_size x sequence_length x vocab_size+1
        input_data = np.reshape(input_data,
                                (len(input_data),
                                 sequence_length,
                                 vocab_size+1))
        print("input_data.shape: ",input_data.shape)
        yield input_data, output_data

In [0]:
a = batch_generator()


In [0]:
x, y = next(a)

In [0]:
x.shape

In [0]:
y.shape

### Build the Model

In [0]:
model = tf.keras.models.Sequential()

#LSTM
model.add(tf.keras.layers.LSTM(256, input_shape=(sequence_length,vocab_size+1)))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(vocab_size+1, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy') #No accuracy tracking here

### Print model output during Training

In [0]:
import numpy as np

In [0]:
#Identify a random sequence which we will use to generate output
start_pos = np.random.randint(0, high=(len(book_num) - sequence_length))
test_seq =  book_num[start_pos : start_pos+sequence_length]

In [0]:
start_pos

In [0]:
#Print random starting sequence for prediction
print('Initial sequence is: ')
for i in range (sequence_length):
    print(int_to_char[test_seq[i][0]], end='')

### Prediction function

In [0]:
def predict_seq(epoch, logs):
    
    print('\n\nOutput sequence after epoch ', epoch, ' :')
    
    #Initialize predicted output
    predicted_output = ''
    
    #lets predict 50 next chars
    current_seq = np.copy(test_seq)
    
    for i in range(50):
        current_seq_one_hot = tf.keras.utils.to_categorical(current_seq, num_classes=vocab_size+1)
        
        data_input = np.reshape(current_seq_one_hot,(1,
                                                     current_seq_one_hot.shape[0],
                                                     current_seq_one_hot.shape[1]))
        
        #Get the char int with maximum probability
        predicted_char_int = np.argmax(model.predict(data_input)[0])
        
        if (predicted_char_int != 0):
            
            #Add to the predicted out, convert int to char
            predicted_output = predicted_output + int_to_char[predicted_char_int]
        
        #Update seq with new value at the end
        current_seq = np.roll(current_seq, -1)
        current_seq[current_seq.shape[0]-1] = [predicted_char_int]
    
    print(predicted_output)

### Execute the model

In [0]:
#Create a LabdaCallback to do prediction at end of every epoch
lambda_checkpoint = tf.keras.callbacks.LambdaCallback(on_epoch_end=predict_seq)

#Create a model checkpoint to store model after each epoch if loss reduces
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('char_rnn.h5',
                                                      monitor='loss',
                                                      save_best_only=True)

In [0]:
batch_size = 2000
train_generator = batch_generator(batch_size=batch_size)

#Fit generator
model.fit_generator(train_generator,
                    epochs=10,
                    steps_per_epoch = (len(book_num)- sequence_length)// batch_size,                    
                    callbacks=[model_checkpoint, lambda_checkpoint])